# SQL 작성

In [1]:
import sys
import os
import pandas as pd
from scipy import io
import json
import cv2
from glob import glob
sys.path.append('../../src')
import DISData as DD

## SQL 연결

In [2]:
doUT = DD.SQL()

host:  192.168.0.201
user:  diadmin
password:  ········


SQL connection done


## query 작성

In [3]:
query1 = (f'''CREATE TABLE IF NOT EXISTS frgc_landmark_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(255) NOT NULL,
            img_dir VARCHAR(1000) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(255) NOT NULL,
            label_dir VARCHAR(1000) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            label VARCHAR(20) NOT NULL,
            landmark VARCHAR(1000));''')

In [4]:
query2 = (f'''INSERT INTO frgc_landmark_info 
        (img_name, img_dir, img_format, img_width, img_height,
        color_space, label_name, label_dir, label_format, label,
        landmark)
        VALUES (%s, %s, %s, %s, %s,  %s, %s, %s, %s, %s,  %s)''')

In [5]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [8]:
value_list1 = []
error_list = []

label_path = 'Z:\\frgc\\data\\RGB\\labeldata'
img_path = 'Z:\\frgc\\data\\RGB\\rawdata'
label_lists = os.listdir(label_path)

for cnt, label_list in enumerate(label_lists):
    label_files = os.listdir(os.path.join(label_path,label_list))
    total_cnt = len(label_lists)
    color_space = 'RGB'
    for label_file in label_files:
    
        label_dir = os.path.join(label_path,label_list,label_file)
        label_name, label_format = os.path.splitext(os.path.basename(label_file))
        label_format = label_format.replace('.','')

        # print(label_dir)
        # print(img)
        img_dir = label_dir.replace('pts','jpg').replace('label','raw')
        # print(img_load)

        img_name, img_format = os.path.splitext(os.path.basename(img_dir))
        img_format = img_format.replace('.','')

        # print(img_dir)
        img = cv2.imread(img_dir)
        img_width = img.shape[0]
        img_height = img.shape[1]
    
    
        with open(label_dir,'r',encoding='utf-8') as f:
            config = f.readlines()
        points = []
        # print(config)
        for point in config:

            x,y = map(int,map(float,point.split(' ')))
            points.append((x,y))
        label = 'face_68'
        landmark = ",".join(map(str, [(int(float(x)), int(float(y))) for x,y in points]))
        img_dir = img_dir.replace('Z:','DataBase')
        label_dir = label_dir.replace('Z:','DataBase')
        # print(landmark)
        value = (img_name, img_dir, img_format, img_width, img_height, color_space,
                label_name, label_dir, label_format, label, landmark)
        value_list1.append(value)

        # print(value)
        if len(value_list1) >= 500:
            
            # print(value_list1[0])
            print(f'landmark {cnt}/{total_cnt}')
            doUT.insert_dataset_values(query2, value_list1)
            value_list1 = []

doUT.insert_dataset_values(query2, value_list1)
# print(value_list2)
value_list1 = []
print('done')     

landmark 47/557
insert 500 done
landmark 97/557
insert 500 done
landmark 141/557
insert 500 done
landmark 180/557
insert 500 done
landmark 233/557
insert 500 done
landmark 297/557
insert 500 done
landmark 352/557
insert 500 done
landmark 410/557
insert 500 done
landmark 475/557
insert 500 done
insert 450 done
done
